In [ ]:
from builtins import range

import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.spm as spm  # spm
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.algorithms.modelgen as model  # model specification
from nipype.interfaces.base import Bunch
from nilearn import plotting
import os  # system functions

%matplotlib inline
